# Basic Samples : Agtype mapper for Psycopg2 driver

You can make transactions and queries for PostgreSQL with Psycopg2.

This module enable to mapping agtype to python class(Path, Vertex, Edge)

## Connect to PostgreSQL and agType setting

In [24]:
import psycopg2 
import age

GRAPH_NAME = "test_graph"
conn = psycopg2.connect(host="172.17.0.2", port="5432", dbname="postgres", user="postgres", password="agens")

age.setUpAge(conn, GRAPH_NAME)


# conn.set_session(readonly=True, autocommit=True)


In [26]:
with conn.cursor() as cursor:
    try :
        cursor.execute("""SELECT * from cypher(%s, $$ CREATE (n:Person {name: 'Joe', title: 'Developer'}) $$) as (v agtype); """, (GRAPH_NAME,) )
        cursor.execute("""SELECT * from cypher(%s, $$ CREATE (n:Person {name: 'Smith', title: 'Developer'}) $$) as (v agtype); """, (GRAPH_NAME,))
        cursor.execute("""SELECT * from cypher(%s, $$ 
            CREATE (n:Person {name: 'Tom', title: 'Manager'}) 
            RETURN n
            $$) as (v agtype); """, (GRAPH_NAME,))
        for row in cursor:
            print("CREATED::", row[0])
        
        
        cursor.execute("""SELECT * from cypher(%s, $$ 
            MATCH (a:Person {name: 'Joe'}), (b:Person {name: 'Smith'}) CREATE (a)-[r:workWith {weight: 5}]->(b)
            $$) as (v agtype); """, (GRAPH_NAME,))
        
        cursor.execute("""SELECT * from cypher(%s, $$ 
            MATCH (a:Person {name: 'Smith'}), (b:Person {name: 'Tom'}) CREATE (a)-[r:workWith {weight: 3}]->(b)
            $$) as (v agtype); """, (GRAPH_NAME,))
        
        # When data inserted or updated, You must commit or set session option 'autocommit' True : conn.set_session(autocommit=True)
        conn.commit()
    except Exception as ex:
        print(type(ex), ex)
        # if exception occurs, you must rollback all transaction. 
        conn.rollback()

with conn.cursor() as cursor:
    try:
        print("------- [Select Vertices] --------")
        cursor.execute("""SELECT * from cypher(%s, $$ MATCH (n) RETURN n $$) as (v agtype); """, (GRAPH_NAME,))
        for row in cursor:
            vertex = row[0]
            print(vertex.id, vertex.label, vertex["name"], vertex["title"])
            print("-->", vertex)
            
        print(type(cursor))
        print("------- [Select Paths] --------")
        cursor.execute("""SELECT * from cypher(%s, $$ MATCH p=()-[]->()s RETURN p LIMIT 10 $$) as (v agtype); """, (GRAPH_NAME,))
        for row in cursor:
            path = row[0]
            v1 = path[0]
            e1 = path[1]
            v2 = path[2]
            print(v1.gtype , v1["name"], e1.gtype , e1.label, e1["weight"], v2.gtype , v2["name"])
            print("-->", path)
    except Exception as ex:
        print(type(ex), ex)
        # if exception occurs, you must rollback even though just retrieving.
        conn.rollback()

CREATED:: {label:Person, id:844424930131987, properties:{name: Tom,title: Manager,}}::VERTEX
------- [Select Vertices] --------
844424930131969 Person Joe None
--> {label:Person, id:844424930131969, properties:{name: Joe,}}::VERTEX
844424930131970 Person Smith None
--> {label:Person, id:844424930131970, properties:{name: Smith,}}::VERTEX
844424930131971 Person Joe None
--> {label:Person, id:844424930131971, properties:{name: Joe,}}::VERTEX
844424930131972 Person Smith None
--> {label:Person, id:844424930131972, properties:{name: Smith,}}::VERTEX
844424930131973 Person Joe None
--> {label:Person, id:844424930131973, properties:{name: Joe,}}::VERTEX
844424930131974 Person Smith None
--> {label:Person, id:844424930131974, properties:{name: Smith,}}::VERTEX
844424930131975 Person Joe None
--> {label:Person, id:844424930131975, properties:{name: Joe,}}::VERTEX
844424930131976 Person Smith None
--> {label:Person, id:844424930131976, properties:{name: Smith,}}::VERTEX
844424930131977 Person J

In [3]:
with conn.cursor() as cursor:
    try:
        cursor.execute("""SELECT * from cypher(%s, $$ 
            MATCH p=(a)-[b]->(c) RETURN a.name, label(b), c.name 
            $$) as (a agtype, b agtype, c agtype); """, (GRAPH_NAME,))
        for row in cursor:
            print(row[0], row[1], row[2])
            print("-->", row)
    except Exception as ex:
        print(ex)
        conn.rollback()

Joe workWith Smith
--> ('Joe', 'workWith', 'Smith')
Smith workWith Tom
--> ('Smith', 'workWith', 'Tom')
Joe workWith Smith
--> ('Joe', 'workWith', 'Smith')
Smith workWith Tom
--> ('Smith', 'workWith', 'Tom')


In [4]:
with conn.cursor() as cursor:
    try :
        cursor.execute("""SELECT * from cypher(%s, $$ 
            CREATE (n:Person {name: 'Jack', title: 'Developer', score:-6.45161290322581e+46}) 
            $$) as (v agtype); """, (GRAPH_NAME,) )
        cursor.execute("""SELECT * from cypher(%s, $$ 
            CREATE (n:Person {name: 'John', title: 'Developer'}) 
            $$) as (v agtype); """, (GRAPH_NAME,))

        cursor.execute("""SELECT * from cypher(%s, $$ 
            MATCH (a:Person {name: 'Jack'}), (b:Person {name: 'John'}) 
            CREATE (a)-[r:workWith {weight: 2, numericVal: 46.420000000000::numeric}]->(b)
            $$) as (v agtype); """, (GRAPH_NAME,))
        
        # When data inserted or updated, You must commit or set session option 'autocommit' True : conn.set_session(autocommit=True)
        conn.commit()
    except Exception as ex:
        print(ex)
        conn.rollback()

# When committed, you must make new cursor.
with conn.cursor() as cursor:
    try:
        cursor.execute("""SELECT * from cypher(%s, $$ 
            MATCH p=(a)-[b]->(c) WHERE b.numericVal>0 RETURN a , b, c 
            $$) as (ta agtype, tb agtype, tc agtype); """, (GRAPH_NAME,))
        
        for row in cursor:
            print(row[0]["name"], row[1].id, row[1]["numericVal"], row[2]["name"])
    except Exception as ex:
        print(ex)
        conn.rollback()
    


In [5]:
with conn.cursor() as cursor:
    try:
        cursor.execute("""SELECT * from cypher(%s, $$ 
            MATCH p=(a)-[b]->(c) RETURN p  
            $$) as (v agtype); """, (GRAPH_NAME,))
        for row in cursor:
            path = row[0]
            print(path[0]["name"], path[1].id, path[1]["numericVal"], path[2]["name"])
    except Exception as ex:
        print(ex)
        conn.rollback()

Joe 1125899906842625 None Smith
Smith 1125899906842626 None Tom
Joe 1125899906842627 None Smith
Smith 1125899906842628 None Tom


In [6]:
age.deleteGraph(conn, GRAPH_NAME)
conn.close()

1 2 arr: [3, 4, 5] args: 9 10 11
1 2 arr: 9 args: 10 11
1 2 cols: (,3,,, ,4,,, ,5,) args: 9 10 11
1 2 cols: 9 args: 10 11
